# Population modelling
As in the previous practical, many of the details and parameters from this lab come from the STELLA version at: http://www3.geosc.psu.edu/~dmb53/DaveSTELLA/Population/pop_modeling.htm

We are going to focus here on modelling change in the human population, since humans interactions with the environment have profound effects on both environmental and human processes. The "anthroposphere" is one of Earth's interconnected spheres!

But you should also realise that many of the simple and general population models we explore can apply to any population, so (get excited biologists!) you could use similar models to explore the growth and pressures affecting populations of organisms.

In each case, we greatly simplify things by assuming all individuals make up a single reservoir, without taking into account the fact that people in different age groups or geographical regions (for example) will be subject to different pressures.

## Simple exponential growth
The simplest population model we can imagine consists of one reservoir representing all of the individuals of a population, with a flux in and a flux out. We are interested here in **global** population, so movement around the globe (e.g. immigration) doesn't change the size of the reservoir.

This means the fluxes affecting the reservoir are:
1. Births -- a flux in
2. Deaths -- a flux out.

Both of these will be a function of the total population (***why?***) as well as the birth and death rates (i.e. the percentage of the population being born or dying at any given time).

So the population at any particular time will be the population at the previous time + the number of births that have occurred - the number of deaths that have occurred.

### Our Basic Population model

Let's warm up our modelling muscles today by constructing this simple model. We are going to start from this and adapt throughout the practical, so **MAKE SURE YOU UNDERSTAND WHAT IT IS DOING BEFORE YOU MOVE ON!!**. Use your work from previous weeks to help you out! It would also be a good idea to draw a diagram for your model to get things going.

First, we will define the model (function). For now we will assume the death rate is constant at 10%, but we will make  the birth rate an **argument (input)** to the function.

The total number of births (births per year) will be the birth rate (births per person per year) multiplied by the population. Similarly, the total number of deaths (deaths per year) will be the death rate (deaths per person per year) multiplied by the population.

We will also assume the following:
- the initial population is 100 people
- a time step of 1 year is sufficient
- we are interested in the population change over 100 years

Finally, we need to be a little bit careful at the end. The population can't ever be less than zero, so we make sure to add an <font face=courier color=green>if</font> statement to prevent negative people!

In [ ]:
import numpy
import matplotlib.pyplot as pyplot
%matplotlib inline

# Define a simple population model
def simple_population_model(Total_Time, birth_rate=0.15):
    
    # Variables
    #-----------
    # Total_Time - number of years to run the model
    # birth_rate - rate of new births per year
    # death_rate - rate of deaths per year
    death_rate=0.10
    
    # Set up time array: Use spacing based on time step (1 year) and total run time
    time = numpy.arange(0,Total_Time,step=1)
    
    # Set up population array to track change over time
    Population = numpy.zeros(len(time))
    
    # Set initial condition - Population = 100
    Population[0] = 100
    
    # Loop over time steps
    for i in range(1,len(time)):
        
        # Calculate the number of births that happened over the last year
        #   based on the population at the previous time [i-1]
        Births = birth_rate * Population[i-1]
        
        # Calculate the number of deaths that happened over the last year
        #   based on the population at the previous time [i-1]
        Deaths = death_rate * Population[i-1]
        
        # At each time [i], calculate the population now (at time = i) based on the population
        #   at the previous time [i-1] and how many births and deaths have occurred
        Population[i] = Population[i-1] + Births - Deaths
        
        # Makes sure the population is never less than zero
        if Population[i] < 0:
            Population[i] = 0
        
    return time,Population

## <font color=blue>Complete Exercise 1 now</font>

## S-shaped growth and carrying capacity
Not many populations grow exponentially for very long -- there are various limitations imposed by the world in which the population grows. In many cases, a population will reach what is called the carrying capacity of its ecosystem, and as this capacity is reached, the population dynamics change and approach a steady state (births = deaths). This kind of growth behavior is simpler than you might expect, and results from some basic assumptions.

As an example, let's assume that as the population increases, the population density goes up (i.e., people live closer together), making it easier for diseases to spread, and also resulting in fewer resources per individual, which lowers the health of the average individual and makes them more susceptible to disease. The net effect is that as the population increases, the death rate increases.

In our model, we can represent this by making the death rate a function of the population variable, rather than a constant value. For now, assume that the death rate increases **linearly** from 0 to 1 (100%) as the population increases from 0 to 10,000 people. In other words:<br><br>
***If Population <= 10000***:
$$death\_rate = Population / 10000.0$$
***Otherwise***:
$$death\_rate = 1.0$$

## <font color=blue>Complete Exercise 2 now</font>

## Overshoot and collapse
In the last model, the population found its steady state in a smooth manner; the rate of change decreased as the carrying capacity was approached. But this is not always the case; sometimes the population's growth rate does not decline fast enough as the carrying capacity is reached, leading to overshoot, which is usually followed by a collapse. One way of interpreting this is that the population does not process information about its condition fast enough to change its growth rate.

A classic example comes from World War II. In short, the US Coast Guard introduced 29 reindeer to a small Arctic island to serve as a food source if needed. After the war, the island was abandoned. The reindeer had no predators but abundant food, and their population grew to 6,000 by 1963. However, the island was small and there wasn't enough food to sustain them. Over the next few years, there was a massive die-off, with only 42 reindeer left in 1966 and none by the 1980s. You can read the story in comic form here: http://www.stuartmcmillen.com/comics_en/st-matthew-island/#page-1

To model this kind of system, we need to add a new **reservoir** to our model to represent the resources. Here we will keep this generic, but you can think of it as representing everything finite the population needs to survive (energy, fresh water, arable land for food production, ...). The size of the Resources reservoir will depend on the size of the Population reservoir, and vice versa.

Before we construct the model, let's work through some important ideas and assumptions one by one.

1. **Resources**: Resources will decline based on how many resources each person uses in a given year. Since Resources are generic for now, let's assume each person typically consumes 1 resource per year. We are assuming there are no new sources of resources. So:
$$Resources\_Used = 1 * Population$$

2. **Resources per person**: The death rate will depend on the remaining resources per person. We can easily represent "resources per person" mathematically:
$$resources\_per\_person = \frac{Resources}{Population}$$

3. **Death rate**: We can safely assume that when resources are abundant, the death rate is not affected by the amount of resources. In that state, we can assume death rate is a constant reflecting things like disease and old age. But once the amount of resources gets low, the death percent begins to increase, reflecting the idea that with fewer resources, living conditions will deteriorate, making survival more difficult. To simplify this relationship in our model, we will assume the death rate is 10% when there are at least 9 resources per person, and that below this the death rate increases linearly with decreasing resources according to the following equation. In other words:<br>
**If resources per person are >=9, then**:
$$death\_rate = 0.10$$
**Otherwise**:
$$death\_rate = -0.1 * resources\_per\_person + 1.0$$

4. **Population**: As in our previous models, the population depends on the birth rate and the death rate. The death rate is now dependent on resource availability, but this complexity should be built into the equation for death rate so the equation for population won't need to change.

## <font color=blue>Complete Exercise 3 now</font>

### Replenishing resources
Sticking with the overshoot model, we are going to explore some modifications.

This scenario explores what happens if resources are not finite but instead there is some process that adds to the resources. This can represent the impacts of technology. For example, while there is a finite supply of oil available from fossil fuels, technology that converts biological materials into oil can increase the overall pool.

## <font color=blue>Complete Exercise 4 now</font>

## <font color=purple>Complete Exercise 5 (extra) now</font>